In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
path = './plant-pathology-2021-fgvc8'
train_dir = path + '/train_images'
test_dir = path +  '/test_images' 
df = pd.read_csv(path + '/train.csv')
df.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [3]:
df.labels.value_counts()

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
rust                               1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [4]:
df['labels'].unique()

array(['healthy', 'scab frog_eye_leaf_spot complex', 'scab', 'complex',
       'rust', 'frog_eye_leaf_spot', 'powdery_mildew',
       'scab frog_eye_leaf_spot', 'frog_eye_leaf_spot complex',
       'rust frog_eye_leaf_spot', 'powdery_mildew complex',
       'rust complex'], dtype=object)

In [5]:
df['labels'] = df['labels'].astype(str) 

In [6]:
train_datagen  = ImageDataGenerator(rotation_range = 180,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split = 0.2)
test_datagen  = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

train_generator = train_datagen.flow_from_dataframe(dataframe = df,
                                                   directory = train_dir,
                                                   target_size = (150,150),
                                                   x_col = 'image',
                                                   y_col = 'labels',
                                                   batch_size = 256,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                   subset = 'training')

test_generator = test_datagen.flow_from_dataframe(dataframe = df,
                                                 directory = train_dir,
                                                 target_size = (150,150),
                                                 x_col = 'image',
                                                 y_col = 'labels',
                                                 batch_size = 256,
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

Found 14906 validated image filenames belonging to 12 classes.
Found 3726 validated image filenames belonging to 12 classes.


In [7]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu',padding='same',input_shape=[150,150,3]))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(16,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(16,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(12,activation='softmax'))


In [8]:
from tensorflow.keras.optimizers import RMSprop,Adam
epochs = 10
Batch_size = 64
optimizer = Adam(lr = 0.001)
model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 16)        9

In [9]:
history = model.fit(train_generator,epochs = 1,batch_size=64,validation_data = test_generator)